In [1]:
import os

In [2]:
%pwd

'/Users/manojkumar/xrayMlOps/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/manojkumar/xrayMlOps'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/manojdavuluri/xrayMlOps.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="manojdavuluri"
os.environ["MLFLOW_TRACKING_PASSWORD"]="10b1ebe5ebd2fc409a3832c6347cadba0c0d8484"

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from xRayClassifier.constants import *
from xRayClassifier.utils.common import read_yaml, create_directories, save_json  

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Dataset",
            mlflow_uri="https://dagshub.com/manojdavuluri/xrayMlOps.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config
    
    

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data, 
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()
    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # if tracking_url_type_store != "file":

            #     # Register the model
            #     # There are other ways to use the Model Registry, which depends on the use case,
            #     # please refer to the doc for more information:
            #     # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            #     mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            # else:
            #     mlflow.keras.log_model(self.model, "model")

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation() 
    evaluation.log_into_mlflow()
except Exception as e:
   raise e

[2023-12-04 21:41:09,983: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-04 21:41:09,987: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-04 21:41:09,988: INFO: common: created directory at: artifacts]
Found 88 images belonging to 2 classes.
1/1 [==============================] - 12s 12s/step - loss: 0.8579 - accuracy: 0.5000
[2023-12-04 21:41:23,792: INFO: common: json file saved at: scores.json]


RestException: INVALID_PARAMETER_VALUE: Response: {'error_code': 'INVALID_PARAMETER_VALUE'}

In [14]:
mlflow.__version__

'2.8.1'